# 🌏 百度街景全景图裁剪工具 v2.0
## Baidu Street View Panorama Cropping Tool

基于 **Yin & Wang (2016)** 的方法，将等距柱状投影(Equirectangular)全景图裁剪为三个视角。

### ✨ v2.0 改进
- **基于角度/比例定义裁剪参数**，自动适配任意尺寸的全景图
- 无需固定像素值，代码更简洁、更灵活

### 裁剪参数
| 参数 | 值 | 说明 |
|------|----| ---- |
| 水平视角 | 90° | 每个视角左右各45° |
| 垂直视角 | 55° | 向上45° + 向下10° |
| 三个视角 | Left / Front / Right | 分别朝向 -90° / 0° / +90° |

### 等距柱状投影的关键特性
```
水平方向: 360° = 图像宽度 (100%)
垂直方向: 180° = 图像高度 (100%)
```

---
## 📁 第1步：挂载Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

---
## ⚙️ 第2步：设置文件夹路径

**请修改下方的 `INPUT_FOLDER` 为您的全景图文件夹路径**

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  请修改此处的路径为您的全景图文件夹路径                              ║
# ╚══════════════════════════════════════════════════════════════════╝

# 输入文件夹路径（包含全景图的文件夹）
INPUT_FOLDER = '/content/drive/MyDrive/panorama_images'  # ← 请修改为您的路径

# 输出文件夹路径（裁剪后的图片保存位置）
# 设为 None 则自动在输入文件夹同级创建 '{文件夹名}_cropped' 文件夹
OUTPUT_FOLDER = None

# 支持的图片格式
SUPPORTED_FORMATS = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.JPG', '.JPEG', '.PNG')

### 🔍 查看Google Drive文件夹结构（可选）

In [ ]:
import os

# 列出 MyDrive 根目录
print("Google Drive 根目录内容:")
print("=" * 50)
drive_path = '/content/drive/MyDrive'
if os.path.exists(drive_path):
    for item in sorted(os.listdir(drive_path))[:20]:
        full_path = os.path.join(drive_path, item)
        item_type = "[文件夹]" if os.path.isdir(full_path) else "[文件]"
        print(f"  {item_type} {item}")
    print("\n... (如有更多文件/文件夹未显示)")
else:
    print("请先运行第1步挂载Google Drive")

---
## 📐 第3步：配置裁剪参数（基于角度/比例）

### 核心原理
等距柱状投影全景图的像素与角度有线性对应关系：
- **水平方向**: 360° 对应图像宽度的 100%
- **垂直方向**: 180° 对应图像高度的 100%

因此，我们用**角度**定义裁剪参数，然后根据实际图像尺寸自动计算像素值。

In [ ]:
# ============================================================================
# 裁剪参数配置 - 基于角度/比例 (v2.0)
# ============================================================================

# 视角参数（度）- 这是唯一需要定义的参数！
HORIZONTAL_FOV = 90     # 水平视角: 90° (每个裁剪框的宽度)
VERTICAL_UP = 45        # 向上角度: 45° (从地平线向上)
VERTICAL_DOWN = 10      # 向下角度: 10° (从地平线向下)

# 三个视角的朝向角度 (相对于前方 0°)
VIEW_DIRECTIONS = {
    'left':  -90,  # 左转 90°
    'front':   0,  # 正前方
    'right':  90   # 右转 90°
}

# ============================================================================
# 计算比例参数（基于等距柱状投影的几何特性）
# ============================================================================

# 水平方向：360° = 100% 宽度
CROP_WIDTH_RATIO = HORIZONTAL_FOV / 360.0  # 90° / 360° = 0.25 (25%)

# 垂直方向：180° = 100% 高度，地平线在中央 (90°)
CROP_TOP_RATIO = (90 - VERTICAL_UP) / 180.0    # (90-45)/180 = 0.25 (从顶部算起)
CROP_BOTTOM_RATIO = (90 + VERTICAL_DOWN) / 180.0  # (90+10)/180 = 0.556
CROP_HEIGHT_RATIO = CROP_BOTTOM_RATIO - CROP_TOP_RATIO  # 0.306 (30.6%)

# 每个视角的水平中心位置（比例）
# 全景图中心对应前方 0°，左侧为负，右侧为正
VIEW_CENTER_RATIOS = {
    view: 0.5 + (angle / 360.0)  # 转换为 0-1 的比例
    for view, angle in VIEW_DIRECTIONS.items()
}

# 打印参数信息
print("╔" + "═" * 62 + "╗")
print("║" + " 裁剪参数配置 v2.0 (基于角度/比例)".center(58) + "  ║")
print("╠" + "═" * 62 + "╣")
print(f"║  水平视角 (FOV):     {HORIZONTAL_FOV}°  →  宽度比例 = {CROP_WIDTH_RATIO:.2%}".ljust(63) + "║")
print(f"║  垂直视角:           {VERTICAL_UP}° 上 + {VERTICAL_DOWN}° 下 = {VERTICAL_UP + VERTICAL_DOWN}°".ljust(63) + "║")
print(f"║  垂直裁剪范围:       {CROP_TOP_RATIO:.2%} ~ {CROP_BOTTOM_RATIO:.2%} (高度 {CROP_HEIGHT_RATIO:.2%})".ljust(63) + "║")
print("╠" + "═" * 62 + "╣")
print("║  视角方向:                                                    ║")
for view, angle in VIEW_DIRECTIONS.items():
    center = VIEW_CENTER_RATIOS[view]
    print(f"║    {view.upper():6s}: {angle:+4d}°  →  水平中心 = {center:.2%}".ljust(63) + "║")
print("╠" + "═" * 62 + "╣")
print("║  ✓ 参数已自动计算，可适配任意尺寸的全景图                      ║")
print("╚" + "═" * 62 + "╝")

### 📊 预览不同尺寸下的像素值
演示同一套比例参数如何自动适配不同分辨率的全景图

In [ ]:
# 演示不同尺寸全景图的裁剪像素值
demo_sizes = [
    (2048, 1024, "标准 (1×)"),
    (4096, 2048, "高清 (2×)"),
    (8192, 4096, "超高清 (4×)"),
    (1024, 512, "低分辨率 (0.5×)")
]

print("\n不同尺寸全景图的自动适配演示:")
print("=" * 75)
print(f"{'输入尺寸':^20} {'输出尺寸':^15} {'Front区域 (像素)':^30}")
print("-" * 75)

for width, height, label in demo_sizes:
    crop_w = round(width * CROP_WIDTH_RATIO)
    crop_h = round(height * CROP_HEIGHT_RATIO)
    
    center_x = round(width * VIEW_CENTER_RATIOS['front'])
    x_start = center_x - crop_w // 2
    x_end = x_start + crop_w
    y_start = round(height * CROP_TOP_RATIO)
    y_end = round(height * CROP_BOTTOM_RATIO)
    
    print(f"{label:^8} {width}×{height:^10} {crop_w}×{crop_h:^12} x=[{x_start}, {x_end}] y=[{y_start}, {y_end}]")

print("=" * 75)

---
## 🔧 第4步：定义裁剪函数（基于比例）

In [ ]:
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


def calculate_crop_regions(width, height):
    """
    根据图像尺寸和比例参数，计算三个视角的裁剪区域
    
    Parameters:
    -----------
    width : int
        全景图宽度（像素）
    height : int
        全景图高度（像素）
    
    Returns:
    --------
    dict : 包含三个视角裁剪坐标的字典
    """
    # 计算裁剪框尺寸（像素）
    crop_width = round(width * CROP_WIDTH_RATIO)
    
    # 计算垂直裁剪范围（像素）
    crop_top = round(height * CROP_TOP_RATIO)
    crop_bottom = round(height * CROP_BOTTOM_RATIO)
    
    # 计算每个视角的水平裁剪范围
    crop_regions = {}
    for view, center_ratio in VIEW_CENTER_RATIOS.items():
        center_x = round(width * center_ratio)
        x_start = center_x - crop_width // 2
        x_end = x_start + crop_width
        
        # 处理边界环绕（全景图水平方向是循环的）
        # 但在实际裁剪中，通常不需要处理，因为视角设计已避免越界
        crop_regions[view] = {
            'x_start': x_start,
            'x_end': x_end,
            'y_start': crop_top,
            'y_end': crop_bottom
        }
    
    return crop_regions


def crop_panorama(image_path, output_dir, filename_base):
    """
    裁剪单张全景图为三个视角 (left, front, right)
    
    Parameters:
    -----------
    image_path : str
        输入全景图的完整路径
    output_dir : str
        输出目录路径
    filename_base : str
        输出文件的基础名称（不含扩展名）
    
    Returns:
    --------
    dict : 包含处理结果的字典
    """
    results = {
        'success': False,
        'input_file': image_path,
        'output_files': [],
        'input_size': None,
        'output_size': None,
        'error': None
    }
    
    try:
        # 打开图像
        img = Image.open(image_path)
        width, height = img.size
        results['input_size'] = (width, height)
        
        # 根据实际图像尺寸计算裁剪区域（自动适配任意尺寸）
        crop_regions = calculate_crop_regions(width, height)
        
        # 获取原始文件扩展名
        _, ext = os.path.splitext(image_path)
        if not ext:
            ext = '.jpg'
        
        # 裁剪并保存三个视角
        for view_name, region in crop_regions.items():
            # 裁剪区域: (left, top, right, bottom)
            crop_box = (
                region['x_start'],
                region['y_start'],
                region['x_end'],
                region['y_end']
            )
            
            # 执行裁剪
            cropped_img = img.crop(crop_box)
            
            # 记录输出尺寸（只需记录一次）
            if results['output_size'] is None:
                results['output_size'] = cropped_img.size
            
            # 生成输出文件名: 原文件名_视角.扩展名
            output_filename = f"{filename_base}_{view_name}{ext}"
            output_path = os.path.join(output_dir, output_filename)
            
            # 保存图像 (保持高质量)
            if ext.lower() in ['.jpg', '.jpeg']:
                cropped_img.save(output_path, quality=95)
            else:
                cropped_img.save(output_path)
            
            results['output_files'].append(output_path)
        
        results['success'] = True
        
    except Exception as e:
        results['error'] = str(e)
    
    return results


def process_folder(input_folder, output_folder=None):
    """
    批量处理文件夹中的所有全景图
    """
    # 检查输入文件夹
    if not os.path.exists(input_folder):
        print(f"❌ 错误: 输入文件夹不存在!")
        print(f"   路径: {input_folder}")
        return None
    
    # 设置输出文件夹
    if output_folder is None:
        parent_dir = os.path.dirname(input_folder.rstrip('/'))
        folder_name = os.path.basename(input_folder.rstrip('/'))
        output_folder = os.path.join(parent_dir, f"{folder_name}_cropped")
    
    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)
    
    # 获取所有图片文件
    image_files = [f for f in os.listdir(input_folder) 
                   if f.lower().endswith(SUPPORTED_FORMATS)]
    
    if not image_files:
        print(f"⚠️ 警告: 在文件夹中没有找到图片文件")
        print(f"   路径: {input_folder}")
        return None
    
    print(f"📂 输入文件夹: {input_folder}")
    print(f"📁 输出文件夹: {output_folder}")
    print(f"🖼️  找到 {len(image_files)} 张全景图待处理\n")
    
    # 统计结果
    stats = {
        'total': len(image_files),
        'success': 0,
        'failed': 0,
        'failed_files': [],
        'input_size': None,
        'output_size': None
    }
    
    # 批量处理 (带进度条)
    for filename in tqdm(image_files, desc="🔄 处理进度", unit="张"):
        image_path = os.path.join(input_folder, filename)
        filename_base, _ = os.path.splitext(filename)
        
        result = crop_panorama(image_path, output_folder, filename_base)
        
        if result['success']:
            stats['success'] += 1
            # 记录尺寸信息（取第一张成功的）
            if stats['input_size'] is None:
                stats['input_size'] = result['input_size']
                stats['output_size'] = result['output_size']
        else:
            stats['failed'] += 1
            stats['failed_files'].append({
                'file': filename,
                'error': result['error']
            })
    
    # 打印统计结果
    print("\n" + "═" * 55)
    print("✅ 处理完成!")
    print("═" * 55)
    print(f"   总计处理:   {stats['total']} 张全景图")
    print(f"   成功:       {stats['success']} 张")
    print(f"   失败:       {stats['failed']} 张")
    print(f"   生成裁剪图: {stats['success'] * 3} 张 (每张全景图 × 3个视角)")
    
    if stats['input_size'] and stats['output_size']:
        print(f"\n   输入尺寸:   {stats['input_size'][0]} × {stats['input_size'][1]} 像素")
        print(f"   输出尺寸:   {stats['output_size'][0]} × {stats['output_size'][1]} 像素 (每个视角)")
    
    if stats['failed_files']:
        print("\n⚠️ 失败文件:")
        for item in stats['failed_files']:
            print(f"   - {item['file']}: {item['error']}")
    
    print(f"\n📁 输出保存在: {output_folder}")
    print("═" * 55)
    
    return stats

print("✅ 函数定义完成! (v2.0 - 基于比例的裁剪)")

---
## 🚀 第5步：执行裁剪处理

**请确保已在第2步中正确设置了 `INPUT_FOLDER` 路径**

In [ ]:
# ============================================================================
# 执行批量裁剪
# ============================================================================

print("🚀 开始处理全景图 (v2.0 - 基于比例自动适配)...\n")

# 检查路径是否存在
if os.path.exists(INPUT_FOLDER):
    # 执行批量处理
    results = process_folder(INPUT_FOLDER, OUTPUT_FOLDER)
else:
    print("❌ 错误: 找不到输入文件夹!")
    print(f"   当前设置的路径: {INPUT_FOLDER}")
    print("\n💡 提示: 请返回第2步修改 INPUT_FOLDER 变量")

---
## 👀 第6步：预览裁剪结果（可选）

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def preview_results(input_folder, output_folder=None, num_samples=2):
    """
    预览裁剪结果，并在原图上显示裁剪区域
    """
    if output_folder is None:
        parent_dir = os.path.dirname(input_folder.rstrip('/'))
        folder_name = os.path.basename(input_folder.rstrip('/'))
        output_folder = os.path.join(parent_dir, f"{folder_name}_cropped")
    
    if not os.path.exists(output_folder):
        print("❌ 输出文件夹不存在，请先运行第5步")
        return
    
    # 获取原始图片列表
    original_files = [f for f in os.listdir(input_folder) 
                     if f.lower().endswith(SUPPORTED_FORMATS)][:num_samples]
    
    if not original_files:
        print("没有找到图片文件")
        return
    
    for orig_file in original_files:
        filename_base, ext = os.path.splitext(orig_file)
        
        fig = plt.figure(figsize=(16, 8))
        
        # 显示原始全景图 (上方，跨3列)
        ax_orig = fig.add_subplot(2, 1, 1)
        orig_path = os.path.join(input_folder, orig_file)
        orig_img = Image.open(orig_path)
        width, height = orig_img.size
        ax_orig.imshow(orig_img)
        
        # 在原图上绘制裁剪区域框
        crop_regions = calculate_crop_regions(width, height)
        colors = {'left': '#FF4444', 'front': '#44FF44', 'right': '#4444FF'}
        for view, region in crop_regions.items():
            rect = patches.Rectangle(
                (region['x_start'], region['y_start']),
                region['x_end'] - region['x_start'],
                region['y_end'] - region['y_start'],
                linewidth=2, edgecolor=colors[view], facecolor='none', linestyle='--'
            )
            ax_orig.add_patch(rect)
        
        ax_orig.set_title(f'Original Panorama: {orig_file}\n{width}×{height} pixels (裁剪区域已标注)', 
                         fontsize=12, fontweight='bold')
        ax_orig.axis('off')
        
        # 显示三个裁剪视角 (下方)
        views = ['left', 'front', 'right']
        titles = ['LEFT (-90°)', 'FRONT (0°)', 'RIGHT (+90°)']
        
        for idx, (view, title) in enumerate(zip(views, titles)):
            ax = fig.add_subplot(2, 3, idx + 4)
            crop_path = os.path.join(output_folder, f"{filename_base}_{view}{ext}")
            
            if os.path.exists(crop_path):
                crop_img = Image.open(crop_path)
                ax.imshow(crop_img)
                ax.set_title(f'{title}\n{crop_img.size[0]}×{crop_img.size[1]} px', 
                           fontsize=11, color=colors[view], fontweight='bold')
            else:
                ax.text(0.5, 0.5, 'Not Found', ha='center', va='center', fontsize=12)
            ax.axis('off')
        
        plt.tight_layout()
        plt.show()
        print("\n" + "-" * 60 + "\n")

# 预览前2张图片的裁剪结果
preview_results(INPUT_FOLDER, OUTPUT_FOLDER, num_samples=2)

---
## 📋 使用说明

### v2.0 改进说明
| 版本 | 参数定义方式 | 适配性 |
|------|------------|--------|
| v1.0 | 固定像素值 + 缩放逻辑 | 需要检测尺寸并计算缩放 |
| **v2.0** | **角度/比例** | **自动适配任意尺寸** |

### 核心参数（仅需修改这些）
```python
HORIZONTAL_FOV = 90   # 水平视角（度）
VERTICAL_UP = 45      # 向上角度（度）
VERTICAL_DOWN = 10    # 向下角度（度）
```

### 自动计算的比例
- 裁剪宽度 = `HORIZONTAL_FOV / 360` = 25%
- 裁剪高度 = `(VERTICAL_UP + VERTICAL_DOWN) / 180` ≈ 30.6%

### 文件命名规则
裁剪后的图片文件名格式为：`原文件名_视角.扩展名`

例如，原文件 `street_001.jpg` 将生成：
- `street_001_left.jpg` - 左侧视角 (-90°)
- `street_001_front.jpg` - 前方视角 (0°)
- `street_001_right.jpg` - 右侧视角 (+90°)

### 参考文献
- Yin, L., & Wang, Z. (2016). Measuring visual enclosure for street walkability. *Applied Geography*, 76, 147-153.